# How to use the generator utility

This tool allows to generate synthetic photometry in a set of available systems from the input internally-calibrated continuously-represented mean spectra.

In [ ]:
# Import the tool and the PhotometricSystem class
from gaiaxpy import generate, PhotometricSystem

We can see which systems are available.

In [ ]:
PhotometricSystem.get_available_systems()

!!! **HERE I would add an explanation of regular vs standardised systems.**

## Basic usage

#### The generator returns a single output: 

 1. A DataFrame of all synthetic photometry results. The DataFrame contains the magnitudes, fluxes and flux errors for each band in the system.

In [ ]:
# Path to file with XP CONTINUOUS RAW data (csv, ecsv, fits, or xml)
f = '/path/to/XP_CONTINUOUS_RAW.xml'
# Select a photometric system
phot_system = PhotometricSystem.Gaia_DR3_Vega
synthetic_photometry = generate(f, photometric_system=phot_system)
synthetic_photometry

#### A list of PhotometricSystem objects is also allowed.

In [ ]:
# Create a list of photometric systems
phot_system_list = [PhotometricSystem.Gaia_DR3_Vega, PhotometricSystem.JKC, PhotometricSystem.Pristine]
synthetic_photometry = generate(f, photometric_system=phot_system_list)
synthetic_photometry

### Input types

The input does not have to be a path to a file. There are other options: a pandas DataFrame, an ADQL query or a list of sourceIds.

#### Passing a DataFrame

In [ ]:
import pandas as pd

f = '/path/to/XP_CONTINUOUS_RAW.csv'
df = pd.read_csv(f) # The values in the DataFrame can be edited if the user wishes to do so.

synthetic_photometry = generate(df, photometric_system=PhotometricSystem.Els_Custom_W09_S2)
synthetic_photometry

#### Running a query

**Important note**: Currently the data in the Gaia Archive is not public. The following code will <span style="color:red">not</span> work until DR3.

In [ ]:
query_input = "select TOP 2 source_id from gaiadr3.gaia_source where has_xp_continuous = 'True'"
phot_system_list = [PhotometricSystem.Stromgren, PhotometricSystem.Stromgren_Std]
synthetic_photometry = generate(query_input, photometric_system=phot_system_list)
synthetic_photometry

#### Passing a list

A list of sourceIds can be passed to the generator as the first argument. The generator will then query the Archive for these objects.

In [ ]:
sources_list = ['48', 44] # The sourceIds can be string or long.
synthetic_photometry = generate(sources_list, photometric_system=PhotometricSystem.JKC_Std)
synthetic_photometry

## Advanced usage

The generator contains shortcuts for two other tools:

1. Application of the colour equation
2. Error correction

#### Colour equation

!!! **HERE: Missing description**

In [ ]:
# To apply the colour equation, first create a list of systems.
phot_system_list = [PhotometricSystem.SDSS_Std, PhotometricSystem.JKC_Std, PhotometricSystem.Pristine]
synthetic_phot_colour = generate(df, photometric_system=phot_system_list, colour_equation=True)
synthetic_phot_colour

We can verify that, for the SDSS_Std system only the u band was affected by the colour equation. (The same can be done for JKC_Std and the U band.)

In [ ]:
# To compare, we compute the photometry for the same systems without applyting the colour equation.
synthetic_phot_regular = generate(df, photometric_system=phot_system_list) # False is the default value

In [ ]:
# Get the bands of both systems
SDSS_Std_bands = PhotometricSystem.SDSS_Std.get_bands()
JKC_Std_bands = PhotometricSystem.JKC_Std.get_bands()

In [ ]:
# Now we can get the columns that correspond to Sdss_Std system for both DataFrames
SDSS_columns = [column for column in synthetic_phot_colour.columns if column.startswith('Sdss')]
SDSS_Std_colour = synthetic_phot_colour[SDSS_columns]
SDSS_Std_regular = synthetic_phot_regular[SDSS_columns]

By running the following command we can see the differences between both DataFrames:

In [ ]:
import pandas.testing as pdt
try:
    pdt.assert_frame_equal(SDSS_Std_colour, SDSS_Std_regular)
except AssertionError as err:
    print(err)

If the colour equation is applied over a system for which this correction is not available, no changes will be made to this system.

In [ ]:
# The first system will change, the second will not.
phot_system_list = [PhotometricSystem.SDSS_Std, PhotometricSystem.IPHAS]
synthetic_phot_regular = generate(df, photometric_system=phot_system_list)
synthetic_phot_colour = generate(df, photometric_system=phot_system_list, colour_equation=True)
# The outputs are different
print(f'Are the outputs equal? {synthetic_phot_colour.equals(synthetic_phot_regular)}')
sdss_columns = [column for column in synthetic_phot_colour.columns if column.startswith('Sdss')]
print(f'Are the SDSS columns equal? {synthetic_phot_colour[sdss_columns].equals(synthetic_phot_regular[sdss_columns])}')
iphas_columns = [column for column in synthetic_phot_colour.columns if column.startswith('Iphas')]
print(f'Are the IPHAS columns equal? {synthetic_phot_colour[iphas_columns].equals(synthetic_phot_regular[iphas_columns])}')

#### Error correction

!!! **HERE: Missing description**

In [ ]:
# To apply the error correction, first create a list of systems.
phot_system_list = [PhotometricSystem.Euclid_VIS, PhotometricSystem.Els_Custom_W09_S2]
synthetic_phot_corrected = generate(df, photometric_system=phot_system_list, error_correction=True)
synthetic_phot_corrected
# A warning is raised for each of the systems that cannot be corrected. No changes are made to these systems.

#### Colour equation and error correction

Both options colour_equation and error_correction can be True at the same time. The error correction needs the Gaia_DR3_Vega system to be present in the input data, so the system is internally added and then removed before returning the results.

In [ ]:
phot_system_list = [PhotometricSystem.Euclid_VIS, PhotometricSystem.Els_Custom_W09_S2]
synthetic_phot_corrected = generate(df, photometric_system=phot_system_list, error_correction=True)
synthetic_phot_corrected
# A warning is raised for each of the systems that cannot be corrected. No changes are made to these systems.

#### Additional parameters

Additional arguments can be passed to the generator.

These are:
1. output_path
2. output_file
3. output_format
4. save_file

Three parameters: **output_path**, **output_file**, and **output_format** define the entire path of the resulting file.

The default output path is the current path. If the given output path does not exist, it will be created.

The default output file name is 'output_spectra'. 

The default output format is the format of the input file (i.e. if the input file is a 'fits', then the output file will be a FITS file by default.), or CSV in any other case (DataFrame, ADQL query or list).

**NOTE: If a file with the same path and name already exists, it will be AUTOMATICALLY OVERWRITTEN.**

In [ ]:
synthetic_photometry = generate(f, photometric_system=phot_system, output_path='/path/to', output_file='my_file', output_format='xml')

The additional parameter **save_file** is a boolean that tells the program whether to save the results or not.
If 'output_file' is given but 'save_file' is set to False, a warning will be raised.

In [ ]:
synthetic_photometry = generate(f, photometric_system=phot_system, output_file='my_file', save_file=False)